In [46]:
from bs4 import BeautifulSoup
from selenium import webdriver
from html.parser import HTMLParser
import html
import requests
import json
from time import sleep
from urllib.parse import quote
import re
import pandas as pd

In [16]:
url = 'https://www.chitai-gorod.ru/search/result.php?q={}&type=author'
authors = ['Фрай М.', 'Хантер Э.', 'Емец Д.']

In [6]:
author_url = [url.format(
            quote(i, encoding='windows-1251')
        ) for i in authors]

In [35]:
def get_id(driver):
    sleep(1)
    result = []
    items = driver.find_elements_by_class_name('product-card')
    result = [item.get_attribute('data-product') for item in items]
    return result

def get_books_info(driver, url):
    driver.get(url)
    sleep(5)
    card_count = int(
              driver.find_element_by_class_name(
                  'count-result__value'
              ).text)
    while len(driver.find_elements_by_class_name(
                    'product-card')) != card_count:
        driver.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);")
        elem = driver.find_element_by_css_selector(
            '.js__show-more-cards')
        try:
            elem.click()
        except:
            pass
        sleep(2)
    return get_id(driver)

a_books = []
with webdriver.Safari() as driver:
    for i in author_url:
        a_books += get_books_info(driver, i)

In [20]:
def get_items(soup):
    res = []
    for i in soup.find_all('div', class_="product-prop__value"):
        res.append(' '.join(re.findall('\S+', i.text)))
    return res

def get_keys(soup):
    res = []
    for i in soup.find_all('div', class_="product-prop__title"):
        attr = ' '.join(re.findall('\S+', i.text))
        if attr == 'ID товара:':
            res.append('ID товара')
        else:
            res.append(attr)
    return res

def extract_book_info(card_id):
    url = 'https://www.chitai-gorod.ru/catalog/book/'
    book_html = requests.get(url+card_id).text
    soup = BeautifulSoup(book_html, 'html.parser')
    rate = soup.select_one('.rating').text.split()
    price = soup.find('div', class_="price")
    image = soup.select_one('.product__image')
    buf = {
        "ID карточки": int(card_id),
        "Название": soup.find('h1', class_='js-analytic-product-title').text,
        "Автор": ' '.join(re.findall('\S+', soup.find('a', class_='product__author').text)),
        "Рейтинг": float(rate[0]),
        "Голоса": int(re.search('\d+', rate[1]).group(0)),
        }
    if image is not None:
        buf['Обложка'] = image.find('img')['src']
    if price is not None:
        buf['Цена'] = int(re.search('\d+', price.text).group(0))
    buf.update(zip(get_keys(soup), get_items(soup)))
    return buf

In [52]:
result = list(map(extract_book_info, a_books))

df = pd.DataFrame(result)
df.sort_values(by='ID карточки', inplace=True)

In [55]:
df.head()

,ID карточки,ID товара,ISBN,Автор,Возраст,Год издания,Голоса,Издательство,Кол-во страниц,Название,Обложка,Переводчик,Редактор,Рейтинг,Серия,Тип обложки,Тираж,Формат,Художник,Цена
0,244296,1400705,9785367009781,Фрай М.,NaN,2009,0,"Амфора, ООО",478,"Книга для таких, как я",https://img-gorod.ru/upload/iblock/73b/73b0845...,NaN,NaN,0.0,ФРАМ,Твердая бумажная,4000,20.5 x 13.2 x 3,NaN,NaN
1,245728,1665067,9785942783372,Фрай М.,NaN,2010,0,"Амфора, ООО",606,Волонтеры вечности,https://img-gorod.ru/upload/iblock/bd5/bd5917e...,NaN,NaN,0.0,NaN,Твердая бумажная,3000,20.7 x 13.8 x 3.8,NaN,NaN
254,246061,1668112,9785699110018,Емец Д.,NaN,2003,0,Эксмо-Пресс,384,Таня Гроттер и Золотая Пиявка,https://img-gorod.ru/upload/iblock/236/236a031...,NaN,NaN,0.0,Таня Гроттер,Твердая бумажная,NaN,20.6 x 13.5 x 2.3,NaN,NaN
2,246246,1801350,5942783535,Фрай М.,NaN,2003,0,"Амфора, ООО",350,Простые волшебные вещи,https://img-gorod.ru/upload/iblock/82d/82d0b11...,NaN,NaN,0.0,Лабиринты Эхо,Твердая бумажная,NaN,20.8 x 13.7 x 2.3,NaN,NaN
3,246247,1801351,9785367016239,Фрай М.,NaN,2009,0,"Амфора, ООО",367,Темная сторона,https://img-gorod.ru/upload/iblock/a10/a1063f0...,NaN,NaN,0.0,Лабиринты Ехо,Твердая бумажная,3000,20.6 x 13.5 x 2.3,NaN,NaN


In [58]:
with open('./hw_4.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)